In [1]:
import numpy as np
from lfsr import LFSR
import itertools
from math import sin,tan,atan;

In [2]:
# First one is x^17 + x^13 + x^12 + x^10 + x^7 + x^4 + x^2 + x^1 + 1
polys = [0x0001D258, 0x00017E04,
         0x0001FF6B, 0x00013F67,
         0x0001B9EE, 0x000198D1,
         0x000178C7, 0x00018A55,
         0x00015777, 0x0001D911,
         0x00015769, 0x0001991F,
         0x00012BD0, 0x0001CF73,
         0x0001365D, 0x000197F5,
         0x000194A0, 0x0001B279,
         0x00013A34, 0x0001AE41,
         0x000180D4, 0x00017891,
         0x00012E64, 0x00017C72,
         0x00019C6D, 0x00013F32,
         0x0001AE14, 0x00014E76,
         0x00013C97, 0x000130CB,
         0x00013750, 0x0001CB8D]

# Expressed using a 48 MHz clock
periods = [959000, 957000,
           953000, 949000,
           947000, 943000,
           941000, 939000,
           937000, 929000,
           919000, 911000,
           907000, 901000,
           893000, 887000]

In [22]:
def nth(iterable, nth):
    return next(itertools.islice(iterable, nth, nth+1))

def search(poly, cipher):
    i = 0
    lfsr = LFSR(poly)
    while (next(lfsr) != cipher): i += 1
    return i

# cipher captured to test
cipher1 = 125180;
cipher2 = 37370;

candidates = np.zeros((32,3),dtype=int);

for k in range(0,32):
    candidates[k,0] = k;
    candidates[k,2] = search(polys[k], cipher1);
    candidates[k,1] = abs(candidates[k][2] - search(polys[k], cipher2));
    
candidates = sorted(candidates,key=lambda x: x[1]); 

# Can be done in one line :
#candidates = sorted([(k,abs(search(poly, cipher1) - search(poly, cipher2)))for k, poly in enumerate(polys)],key=lambda x: x[1])

In [23]:
lfsr_found = candidates[0][0];
delta      = candidates[0][1];
offset     = candidates[0][2];
print('Delta :',delta);
print('LFSR  :',lfsr_found);

offset1 = offset;
offset2 = offset+delta;
print('Offset1 :',offset1);
print('Offset2 :',offset2);

Delta : 444
LFSR  : 1
Offset1 : 76838
Offset2 : 77282


In [13]:
u = nth(LFSR(polys[lfsr_found]), 37138);
v = nth(LFSR(polys[lfsr_found]), 37607);

if(u==cipher1):
    print(u,': Offset1 ok');
else:
    print(u,': Offset1 not ok');
    
if(v==cipher2):
    print(v,': Offset2 ok');
else:
    print(v,': Offset2 not ok');

115834 : Offset1 not ok
100785 : Offset2 not ok


In [32]:
print(search(polys[1], 73630));

76843


In [33]:
firstBeam = [37138,37145,37145,
             37607,37603,37590,
             37466,37468,37486];

secondBeam = [76838,76838,76843,
              77282,77291,77300,
              77423,77423,77426];

In [34]:
azimuth = np.zeros((9,1),dtype=float);
elevation = np.zeros((9,1),dtype=float);

In [35]:
for i in range(0,9):
    firstBeam[i]  = ((firstBeam[i]  * 8.0) / periods[0]) * 2 * np.pi;
    secondBeam[i] = ((secondBeam[i] * 8.0) / periods[0]) * 2 * np.pi;

    azimuth[i] = ((firstBeam[i] + secondBeam[i]) / 2) - np.pi;
    p = 60/180*np.pi;
    beta = (secondBeam[i] - firstBeam[i]) - 120/180*np.pi;
    elevation[i] = atan(sin(beta/2)/tan(p/2));

In [62]:
print('Angle en degré');
print('');
print('Mesure n°\tAngle\t\tCapteur 1\t\tCapteur 2\t\tCapteur 3');
print('');
print('1\t\tazimuth\t\t',azimuth[0]/np.pi*180,'\t\t',azimuth[1]/np.pi*180,'\t\t',azimuth[2]/np.pi*180);
print('1\t\televation\t',elevation[0]/np.pi*180,'\t\t',elevation[1]/np.pi*180,'\t\t',elevation[2]/np.pi*180);
print('');
print('2\t\tazimuth\t\t',azimuth[3]/np.pi*180,'\t\t',azimuth[4]/np.pi*180,'\t\t',azimuth[5]/np.pi*180);
print('2\t\televation\t',elevation[3]/np.pi*180,'\t\t',elevation[4]/np.pi*180,'\t\t',elevation[5]/np.pi*180);
print('');
print('3\t\tazimuth\t\t',azimuth[6]/np.pi*180,'\t\t',azimuth[7]/np.pi*180,'\t\t',azimuth[8]/np.pi*180);
print('3\t\televation\t',elevation[6]/np.pi*180,'\t\t',elevation[7]/np.pi*180,'\t\t',elevation[8]/np.pi*180);

Angle en degré

Mesure n°	Angle		Capteur 1		Capteur 2		Capteur 3

1		azimuth		 [-8.8577268] 		 [-8.84721585] 		 [-8.83970803]
1		elevation	 [-0.67183363] 		 [-0.69003612] 		 [-0.67703436]

2		azimuth		 [-7.48679875] 		 [-7.47929093] 		 [-7.48529718]
2		elevation	 [-0.73684179] 		 [-0.70303781] 		 [-0.64582979]

3		azimuth		 [-7.48679875] 		 [-7.48379562] 		 [-7.45226277]
3		elevation	 [-0.00346771] 		 [-0.00866928] 		 [-0.04768105]
